<a href="https://colab.research.google.com/github/Yuuu553/-/blob/main/%E6%83%85%E7%B7%92%E6%97%A5%E8%A8%98ver10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#版面修改
!pip install gradio snownlp bcrypt
!pip install pytz

import gradio as gr
import bcrypt
import json
import datetime
import pytz
import random
from snownlp import SnowNLP
from google.colab import drive
import os


# 掛載 Google Drive
drive.mount('/content/drive')

# 路徑設置
users_db_path = "/content/drive/My Drive/emotion_diary_users.json"
records_db_path = "/content/drive/My Drive/emotion_diary_records.json"

# 初始化資料檔案（如果不存在）
if not os.path.exists(users_db_path):
    with open(users_db_path, "w") as file:
        json.dump({"users": []}, file)

if not os.path.exists(records_db_path):
    with open(records_db_path, "w") as file:
        json.dump({}, file)

# 全域變數
session_user = None

# 註冊新使用者
def register_user(username, password):
    with open(users_db_path, "r+") as file:
        users_data = json.load(file)
        if any(user["username"] == username for user in users_data["users"]):
            return "使用者名稱已存在！"
        hashed_password = bcrypt.hashpw(password.encode(), bcrypt.gensalt())
        users_data["users"].append({"username": username, "password_hash": hashed_password.decode()})
        file.seek(0)
        json.dump(users_data, file)
        return "註冊成功！"

# 登入使用者
def login_user(username, password):
    global session_user
    with open(users_db_path, "r") as file:
        users_data = json.load(file)
        user = next((u for u in users_data["users"] if u["username"] == username), None)
        if user and bcrypt.checkpw(password.encode(), user["password_hash"].encode()):
            session_user = username
            return f"歡迎，{username}！", gr.update(visible=True), gr.update(visible=False)
    session_user = None
    return "登入失敗，請檢查使用者名稱與密碼！", gr.update(visible=False), gr.update(visible=True)

# 情緒建議選項
advice_options = {
    "positive": ["保持正能量，與朋友分享你的快樂！", "試試新的挑戰，今天的你充滿自信！"],
    "neutral": ["感覺穩定，可以做些自己喜歡的事放鬆！", "運動可能是保持情緒平穩的好選擇！"],
    "negative": ["試著找一個朋友聊聊你的感受。", "做些讓自己平靜的活動，例如冥想或散步。"],
    "very_negative": ["情緒低落時，給自己一些時間休息吧！", "找信任的人傾訴，能幫助你釋放壓力。"]
}

# 根據情緒得分提供建議
def get_random_advice(sentiment_score):
    if sentiment_score >= 0.8:
        return random.choice(advice_options["positive"])
    elif sentiment_score >= 0.6:
        return random.choice(advice_options["positive"])
    elif sentiment_score >= 0.4:
        return random.choice(advice_options["neutral"])
    elif sentiment_score >= 0.2:
        return random.choice(advice_options["negative"])
    else:
        return random.choice(advice_options["very_negative"])
# 設定時區
def get_current_time_in_timezone(timezone="Asia/Taipei"):
    tz = pytz.timezone(timezone)  # 設定時區
    return datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")

# 儲存情緒記錄
def save_record(emotion, note):
    if not session_user:
        return "請先登入！", []

    if not emotion or not note.strip():
        return "請選擇情緒並填寫感受！", []

    # 分析情緒得分
    sentiment_score = SnowNLP(note).sentiments
    advice = get_random_advice(sentiment_score)

    with open(records_db_path, "r+") as file:
        records_data = json.load(file)
        # 確保當前使用者的紀錄存在
        if session_user not in records_data:
            records_data[session_user] = []  # 初始化空清單
        current_time = get_current_time_in_timezone("Asia/Taipei")  # 獲取當前時間，並設定為 Taipei 時區

        record = {
            "emotion": emotion,
            "note": note.strip(),
            "sentiment": round(sentiment_score, 2),
            "advice": advice,
            "time": current_time,  # 使用調整過的時間
        }
        records_data[session_user].append(record)
        file.seek(0)
        file.truncate()  # 確保覆寫檔案
        json.dump(records_data, file, ensure_ascii=False, indent=4)

    return f"記錄已儲存！ 情緒: {emotion}, 得分: {record['sentiment']}，建議: {advice}", format_records(records_data[session_user])

# 格式化記錄，新增索引值
def format_records(user_records):
    return [
        [idx + 1, record["emotion"], record["note"], record["sentiment"], record["advice"], record["time"]]
        for idx, record in enumerate(user_records)
    ]

# 獲取當前使用者的紀錄
def get_user_records():
    if not session_user:
        return "請先登入！", []

    with open(records_db_path, "r") as file:
        records_data = json.load(file)

    user_records = records_data.get(session_user, [])

    # 確保索引值正確生成
    indexed_records = [
        [idx + 1, record["emotion"], record["note"], record["sentiment"], record["advice"], record["time"]]
        for idx, record in enumerate(user_records)
    ]

    return "以下是你的紀錄：", indexed_records

# 刪除指定紀錄
def delete_record(index):
    if not session_user:
        return "請先登入！", []

    with open(records_db_path, "r+") as file:
        records_data = json.load(file)
        user_records = records_data.get(session_user, [])

        if 1 <= index <= len(user_records):  # 將索引調整為從 1 開始
            deleted = user_records.pop(index - 1)  # 轉換為 0 基索引
            file.seek(0)
            file.truncate()  # 確保覆寫檔案
            json.dump(records_data, file, ensure_ascii=False, indent=4)
            return f"已刪除記錄：{deleted['emotion']} - {deleted['note']}", format_records(user_records)

    return "刪除失敗，請檢查索引是否正確。", format_records(records_data.get(session_user, []))

# 清除所有記錄
def clear_records():
    if not session_user:
        return "請先登入！", []

    with open(records_db_path, "r+") as file:
        records_data = json.load(file)
        # 確保當前使用者的紀錄存在
        if session_user in records_data:
            records_data[session_user] = []
        else:
            records_data[session_user] = []  # 初始化空清單，防止未來存取錯誤
        file.seek(0)
        file.truncate()  # 確保清空後的檔案正確覆寫
        json.dump(records_data, file, ensure_ascii=False, indent=4)
    return "所有記錄已清除！", []

# 登出
def logout_user():
    global session_user
    session_user = None
    return "您已登出！", gr.update(visible=False), gr.update(visible=True), []

# Gradio 應用程式
with gr.Blocks() as app:
    # 分頁分組
    with gr.Row():
        login_form = gr.Group(visible=True)  # 登入與註冊表單
        dashboard = gr.Group(visible=False)  # 使用者登入後的儀表板

    # 登入與註冊表單
    with login_form:
        gr.Markdown("# 登入或註冊")
        message_label = gr.Label(label="訊息")
        username_input = gr.Textbox(label="使用者名稱")
        password_input = gr.Textbox(label="密碼", type="password")
        with gr.Row():
            login_button = gr.Button("登入")
            register_button = gr.Button("註冊")

    # 儀表板功能
    with dashboard:
        gr.Markdown("# 情緒紀錄日記")

        # 寫情緒頁面
        with gr.Row():
            with gr.Column():
                gr.Markdown("### 記錄你的情緒")
                emotion_buttons = gr.Radio(["喜", "怒", "哀", "樂"], label="選擇情緒", interactive=True)
                note_input = gr.Textbox(label="寫下你的感受...", lines=4, placeholder="請輸入文字")
                save_button = gr.Button("儲存記錄")

        # 紀錄頁面
        with gr.Row():
            with gr.Column():
                gr.Markdown("### 你的情緒記錄")
                history_display = gr.Dataframe(
                    headers=["索引", "情緒", "感受", "情緒得分", "建議", "時間"],
                    interactive=False
                )
                delete_index_input = gr.Number(label="輸入要刪除的記錄索引", precision=0)
                delete_button = gr.Button("刪除選定記錄")
                refresh_button = gr.Button("刷新記錄")
                clear_button = gr.Button("清除所有記錄")

        # 登出按鈕
        logout_button = gr.Button("登出")

    # 事件處理綁定
    login_button.click(
        login_user,
        inputs=[username_input, password_input],
        outputs=[message_label, dashboard, login_form]
    )
    register_button.click(
        register_user,
        inputs=[username_input, password_input],
        outputs=message_label
    )
    save_button.click(
        save_record,
        inputs=[emotion_buttons, note_input],
        outputs=[message_label, history_display]
    )
    refresh_button.click(
        get_user_records,
        outputs=[message_label, history_display]
    )
    delete_button.click(
        delete_record,
        inputs=delete_index_input,
        outputs=[message_label, history_display]
    )
    clear_button.click(
        clear_records,
        outputs=[message_label, history_display]
    )
    logout_button.click(
        logout_user,
        outputs=[message_label, dashboard, login_form, history_display]
    )

# 啟動應用程式
app.launch()
